## Задание 1. Ошибки в исходных файлах

Что нужно сделать
У нас есть набор исходников. Необходимо выполнить git clone.

Просмотрите конфиг-файлы, dockerfile, docker-compose.yml и найдите все ошибки. Например, в dockerfile в качестве исходника для образа может не быть слоя FROM nginx:alpine/, в других dockerfile может отсутствовать volume /data, порт 6379, пользователи.

В папке web в dockerfile задайте источник python:2.7-stretch, пользователя root, порт 9000.
Подсказка: в большинстве случаев в месте, где надо что-то добавить, стоит коммент через #.
Внесите правки в docker-compose.yml:
в блок services определите сети — frontend и backend;
в блок volume добавьте путь ./web/cron/crontab_todo:/etc/crontab;
в блок consumers добавьте порт 9000;в блок redis build добавьте ./redis и порт 6379.
Когда вы подумаете, что все исправления сделаны, выполните $ docker-compose up.
Если всё сделано корректно, то запустится два экземпляра Redis.


In [5]:
import os
from git import Repo


# Функция для клонирования git-репозитория в указанную папку, если папка еще не существует
def clone_git_repo_if_not_exists(repo_url, parent_folder, repo_name):
    clone_folder = os.path.join(parent_folder, repo_name)
    # Проверяем, существует ли уже директория
    if os.path.isdir(clone_folder):
        return f'Папка "{clone_folder}" уже существует. Действие не требуется.'
    else:
        # Клонируем репозиторий
        Repo.clone_from(repo_url, clone_folder)
        return f"Репозиторий склонирован в {clone_folder}"


# URL репозитория для клонирования
repo_url = "https://github.com/darkbenladan/docker-nginx-python2-db-redis-homework.git"
# Имя репозитория
repo_name = "docker-nginx-python2-db-redis-homework"
# Путь к родительской папке, куда будет склонирован репозиторий
parent_folder = "data"

# Клонируем репозиторий, если папка не существует
result = clone_git_repo_if_not_exists(repo_url, parent_folder, repo_name)
print(result)

Репозиторий склонирован в data/docker-nginx-python2-db-redis-homework


In [6]:
def modify_file(filepath, changes):
    """Читаем файл, вносим изменения и записываем обратно."""
    with open(filepath, "r", encoding="utf-8") as file:
        lines = file.readlines()

    new_lines = []
    for line in lines:
        for old, new in changes.items():
            if old in line:
                line = line.replace(old, new)
        new_lines.append(line)

    with open(filepath, "w", encoding="utf-8") as file:
        file.writelines(new_lines)


def main():
    base_path = "data/docker-nginx-python2-db-redis-homework"

    # Изменения для Dockerfile в каталоге redis
    redis_dockerfile_path = os.path.join(base_path, "redis/Dockerfile")
    modify_file(
        redis_dockerfile_path,
        {"FROM #REDIS": "FROM redis:6.2.6", "#PORT": "EXPOSE 6379"},
    )

    # Изменения для Dockerfile в каталоге consumers
    consumers_dockerfile_path = os.path.join(base_path, "consumers/Dockerfile")
    modify_file(
        consumers_dockerfile_path,
        {
            "FROM python:2.7-stretch": "FROM python:3.8-slim",
            "RUN pip install uwsgi": "RUN pip install --upgrade pip && pip install uwsgi",
        },
    )

    # Изменения для Dockerfile в каталоге nginx
    nginx_dockerfile_path = os.path.join(base_path, "nginx/Dockerfile")
    modify_file(
        nginx_dockerfile_path,
        {
            "FROM python:2.7-stretch": "FROM nginx:stable-alpine",
            "#PORT": "EXPOSE 80 443",
        },
    )

    # Изменения в docker-compose.yml
    docker_compose_path = os.path.join(base_path, "docker-compose.yml")
    modify_file(docker_compose_path, {"#networks:": "networks:", "#PORT": "expose:"})

    print("Изменения успешно внесены.")


if __name__ == "__main__":
    main()

Изменения успешно внесены.
